In [ ]:
from curses.ascii import SO
import json
import os
import sqlite3
from sqlite3 import Error
from re import S
import regex as re
import imagesize

In [ ]:
SORDI_home = "/SORDI"
SORDI_folders = []

In [ ]:
!rm SORDI.sqlite

In [ ]:
conn = None

In [ ]:
def open_json(filepath):
    # Opening JSON file
    f = open(filepath)
    
    data = json.load(f)
    f.close()

    return data

In [ ]:
def get_concrete_path(directory_in_str, pattern):
    directory = os.fsencode(directory_in_str)
    
    ret_value = None

    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.startswith(pattern):
            full_path = os.path.join(os.fsdecode(directory), filename)
            
            ret_value = full_path
    
    return ret_value

In [ ]:
def traverse_sordi_dirs(directory_in_str, conn):
    directory = os.fsencode(directory_in_str)
    
    labels = {}
    frames = {}

    def just_the_number(fname):
        m = re.findall('[a-zA-Z0-9_]+\.', fname)

        if len(m) == 1:
            return m[0]
            
        return None

    def trav (directory):
        traversed = {}

        for subdir, dirs, files in os.walk(directory):
            for file in files:
                filepath = os.fsdecode(os.path.join(subdir, file)) #os.fsdecode(file)

                if filepath.endswith(".json") or filepath.endswith(".jpg"): 
                    #print(os.path.join(os.fsdecode(directory), filepath))
                    numb = just_the_number(filepath)
                    
                    if numb:
                        traversed[numb] = filepath

                else:
                    continue
        return traversed

    frames = trav(os.path.join(directory_in_str, "images"))
    labels = trav(os.path.join(directory_in_str, "labels/json"))

    print(len(frames), len(labels))

    for key in frames:
        w,h = imagesize.get(frames[key])
        #print(w,h)

        label_json = open_json(labels[key])
        #print(label_json)

        print(frames[key], labels[key])
        
        create_training_frame(conn, (frames[key], json.dumps(label_json), w,h, 0.0))

        pass

In [ ]:
def create_connection(db_file):

    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)

    return conn

In [ ]:
def create_sql_tables(conn):
    
    table = """ CREATE TABLE FRAME (
            path_to_file VARCHAR(255) NOT NULL,
            labels json,
            width INT,
            height INT,
            uncertainty_score FLOAT
        ); """
    
    cur = conn.cursor()
    cur.execute(table)
    conn.commit()

    return cur.lastrowid

In [ ]:
def create_training_frame(conn, task):
    
    sql = ''' INSERT INTO FRAME(path_to_file, labels, width, height, uncertainty_score)
              VALUES(?,?,?,?,?) '''
    
    cur = conn.cursor()
    cur.execute(sql, task)
    #conn.commit()

    return cur.lastrowid

In [ ]:
def create_label_db():

    conn = create_connection("SORDI.sqlite")

    try:
        create_sql_tables(conn)
    except Error as e:
        print(e)

    data = open_json('/SORDI/config.json')

    for i in data:
        #print(i['dataset_id'])
        #SORDI_folders.append(i['dataset_id'])
        SORDI_folders.append(i)

    for f in SORDI_folders:
        ret = get_concrete_path(SORDI_home, f['dataset_id'])

        if ret:
            print(f)
            print(ret)

            traverse_sordi_dirs(ret, conn)

        else:
            print("Missing directory - no match found for {}".format(f['dataset_id']))

    conn.commit()
    conn.close()

In [ ]:
create_label_db()

In [ ]:
!ls